# Digital Speech Processing
### Prof. Dr. Rodrigo Guido

### Student: Álvaro Leandro Cavalcante Carneiro

# Short Test Challenge
Creating and normalizing a band-stop filter.

## Importing the necessary libraries

In [1]:
import math
import numpy as np

## Defining filter parameters
First of all, it's important to define the necessary parameters, like the filter order, samples per second and cutoff frequency. In this case, we have frequency A and B to represent the cutpoints of our band-stop filter.

In [2]:
filter_order = 4
samples_second = 10000
cutoff_freq_a = 2500
cutoff_freq_b = 3500

## Function to get low pass filter
The function above gets the low pass filter. It also accepts a parameter to determine if the filter is a low pass to be transformed into a high pass, to change the division coefficient. If there's a division by 0, the value of the coefficient is replace by 1.

In [3]:
def get_low_pass_filter(filter_order, samples_second, cutoff_frequency, high_pass=False):
    frequency = samples_second / 2
    divisor = frequency / 1000
    dividend = cutoff_frequency / 1000 
    
    if high_pass:
        dividend = divisor - dividend

    final_filter = []

    for n in range(filter_order + 1):
        first_part = math.sin( ((dividend * math.pi) / divisor) * (n - (filter_order/2)) )
        second_part = (math.pi * (n - (filter_order/2)))
        if second_part == 0:
            value = 1
        else:
            value = first_part / second_part
        final_filter.append(value)
    
    return np.array(final_filter)

## Normalizing results
The function above normalize the results to get a gain of 0 dB.

In [4]:
def normalized_coef(coef):
    print('Sum befone normalization', sum(coef))
    normalized = [x/sum(coef) for x in coef]
    print('Sum after normalization', sum(normalized))
    return normalized

Let's get the normalized version of low pass filter, considering the first cutoff frequency.

In [16]:
low_pass_filter = normalized_coef(
    get_low_pass_filter(filter_order, samples_second, cutoff_freq_a, False))
low_pass_filter

Sum befone normalization 1.6366197723675815
Sum after normalization 1.0


[1.1909216478792039e-17,
 0.19449226482417134,
 0.6110154703516573,
 0.19449226482417134,
 1.1909216478792039e-17]

After that, we call the same function, but this time passing the second cutoff frequency, and with the "high_pass" parameter set to True, to change the equation. Again, we can normalize the outputs to get 0 dB in sum of coefficients

In [17]:
low_pass_b = normalized_coef(
    get_low_pass_filter(filter_order, samples_second, cutoff_freq_b, True))
low_pass_b

Sum befone normalization 1.8177669062567468
Sum after normalization 1.0


[0.08326994248114676,
 0.14166728776603074,
 0.550125539505645,
 0.14166728776603074,
 0.08326994248114676]

We'll use the same function created in the last short test to reverse the signal of our low pass filter, to achieve the high pass version! 

In [7]:
def get_high_pass_filter(filter_value):      
    reversed_filter = np.array(filter_value)[::-1]
    inverse = []
    for i, value in enumerate(reversed_filter):
        if i % 2 != 0:
            inverse.append(-value)
        else:
            inverse.append(value)
       
    return np.array(inverse)

In [18]:
high_pass = normalized_coef(get_high_pass_filter(low_pass_b))
high_pass

Sum befone normalization 0.4333308489358771
Sum after normalization 0.9999999999999999


[0.19216250743659558,
 -0.3269263845717899,
 1.2695277542703884,
 -0.3269263845717899,
 0.19216250743659558]

## Obtaining the band-stop filter
To get the band-stop filter, we just need to sum the coefficients of the low and high pass filters calculated previously. 

In [23]:
stop_band = normalized_coef(np.array(low_pass_filter) + np.array(high_pass))
list(stop_band)

Sum befone normalization 1.9999999999999996
Sum after normalization 1.0000000000000002


[0.09608125371829782,
 -0.0662170598738093,
 0.9402716123110231,
 -0.0662170598738093,
 0.09608125371829782]